<a href="https://colab.research.google.com/github/mathengem/Algorithmic-Trading-Backtesting-in-python/blob/main/algotradeLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yahoo_fin
!pip install alpaca_trade_api
!pip install yfinance
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=8c9ebd8163e2a918ef64916e5f9dfd22d44cc82304781ce2cdf937c4107a1b64
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.8
    Uninstalling msgpack-1

In [2]:
!pip install tensorflow

In [4]:
import numpy as np
import pandas as pd
import datetime as dt
import tensorflow as tf
from yahoo_fin import stock_info as yf
from sklearn.preprocessing import MinMaxScaler
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
import requests as rq
import alpaca_trade_api as api
import logging
import json
from multiprocessing import Process, Manager
import time
from datetime import datetime, time
import pytz

# Load configuration from a JSON file
with open('config.json') as config_file:
    config = json.load(config_file)

# SETTINGS
TRADER_BOT_NAME = config['TRADER_BOT_NAME']
TRADER_API_KEY = config['TRADER_API_KEY']
TRADER_API_SECRET = config['TRADER_API_SECRET']
TRADER_API_URL = config['TRADER_API_URL']
TELEGRAM_URL = config['TELEGRAM_URL']
TELEGRAM_BOT_ID = config['TELEGRAM_BOT_ID']
TELEGRAM_CHAT_ID = config['TELEGRAM_CHAT_ID']
SCREENER_INTERVAL = config['SCREENER_INTERVAL']
SCREENER_PERIOD = config['SCREENER_PERIOD']
SCREENER_NASDAQ_COUNT = config['SCREENER_NASDAQ_COUNT']
TAKE_PROFIT_DELTA = config['TAKE_PROFIT_DELTA']
CASH_LIMIT = config['CASH_LIMIT']

N_STEPS = 7
LOOKUP_STEPS = [1, 2, 3]
BATCH_SIZE = 8
EPOCHS = 80

market_open_time = datetime.strptime(config['MARKET_OPEN_TIME'], "%H:%M").time()
market_close_time = datetime.strptime(config['MARKET_CLOSE_TIME'], "%H:%M").time()
timezone = pytz.timezone(config['TIMEZONE'])

# Set up logging
logging.basicConfig(filename='trading_bot.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Performance tracking
performance = {
    'trades': 0,
    'successful_trades': 0,
    'failed_trades': 0,
    'total_profit_loss': 0
}

# Create a shared dictionary for predictions
manager = Manager()
shared_predictions = manager.dict()

def send_message(message):
    try:
        response = rq.post(
            f'{TELEGRAM_URL}/{TELEGRAM_BOT_ID}/sendMessage?chat_id={TELEGRAM_CHAT_ID}&parse_mode=Markdown&text={message}'
        )
        response.raise_for_status()
        return response
    except rq.RequestException as e:
        logging.error(f"Failed to send Telegram message: {str(e)}")
        return None

def get_stock_data(ticker, start_date, end_date):
    try:
        df = yf.get_data(ticker, start_date=start_date, end_date=end_date, interval='1d')
        df = df[['close']]
        df['date'] = df.index
        return df
    except Exception as e:
        logging.error(f"Error fetching data for {ticker}: {e}")
        return None

def prepare_data(df, days):
    scaler = MinMaxScaler()
    df['close'] = scaler.fit_transform(df[['close']])

    df['future'] = df['close'].shift(-days)
    last_sequence = np.array(df[['close']].tail(days))
    df.dropna(inplace=True)

    sequence_data = []
    sequences = deque(maxlen=N_STEPS)

    for entry, target in zip(df[['close', 'date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])

    last_sequence = list([s[:1] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y, scaler

def create_model(input_shape):
    model = Sequential([
        LSTM(60, return_sequences=True, input_shape=input_shape),
        Dropout(0.3),
        LSTM(120, return_sequences=False),
        Dropout(0.3),
        Dense(20),
        Dense(1)
    ])
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def train_model(model, X, Y):
    model.fit(X, Y, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)
    return model

def predict_prices(model, last_sequence, scaler):
    last_sequence = last_sequence[-N_STEPS:]
    last_sequence = np.expand_dims(last_sequence, axis=0)
    prediction = model.predict(last_sequence)
    predicted_price = scaler.inverse_transform(prediction)[0][0]
    return round(float(predicted_price), 2)

def run_predictions(shared_predictions):
    while True:
        for stock in ['GOOGL', 'AAPL', 'MSFT', 'AMZN', 'FB']:  # Add more stocks as needed
            end_date = dt.date.today().strftime('%Y-%m-%d')
            start_date = (dt.date.today() - dt.timedelta(days=1104)).strftime('%Y-%m-%d')

            df = get_stock_data(stock, start_date, end_date)
            if df is None:
                continue

            predictions = []
            for step in LOOKUP_STEPS:
                df_processed, last_sequence, X, Y, scaler = prepare_data(df.copy(), step)
                X = X[:, :, :1].astype(np.float32)

                model = create_model((N_STEPS, 1))
                model = train_model(model, X, Y)

                prediction = predict_prices(model, last_sequence, scaler)
                predictions.append(prediction)

            shared_predictions[stock] = predictions
            logging.info(f"Updated predictions for {stock}: {predictions}")

        time.sleep(3600)  # Wait for an hour before updating predictions again

def CheckStock(stock):
    data = {}
    try:
        df = yf.get_data(stock, period=SCREENER_PERIOD, interval=SCREENER_INTERVAL)
        if len(df) > 0:
            df['RSI'] = ta.rsi(df['Close'], timeperiod=14)
            bbands = ta.bbands(df['Close'], length=20, std=2.3)
            df['L'] = bbands['BBL_20_2.3']
            df['M'] = bbands['BBM_20_2.3']
            df['U'] = bbands['BBU_20_2.3']

            previous2_bar = df[-3:].head(1)
            previous_bar = df[-2:].head(1)
            current_bar = df[-1:]

            if current_bar['RSI'].values[0] > 70 and current_bar['Close'].values[0] > current_bar['U'].values[0]:
                data = {
                    'direction': 'DOWN',
                    'stock': stock,
                    'stop_loss': round(max(previous_bar['High'].values[0], previous2_bar['High'].values[0], previous_bar['U'].values[0]), 2),
                    'take_profit': round(min(previous_bar['Low'].values[0], previous2_bar['Low'].values[0], previous_bar['M'].values[0]), 2)
                }
            elif current_bar['RSI'].values[0] < 30 and current_bar['Close'].values[0] < current_bar['L'].values[0]:
                data = {
                    'direction': 'UP',
                    'stock': stock,
                    'stop_loss': round(min(previous_bar['Low'].values[0], previous2_bar['Low'].values[0], previous_bar['L'].values[0]), 2),
                    'take_profit': round(max(previous_bar['High'].values[0], previous2_bar['High'].values[0], previous_bar['M'].values[0]), 2)
                }
    except Exception as e:
        logging.error(f"Error checking stock {stock}: {str(e)}")
    return data

def is_market_open():
    now = datetime.now(timezone)
    current_time = now.time()
    return (market_open_time <= current_time <= market_close_time) and (now.weekday() < 5)

def ScreenStocks(trader_api):
    try:
        assets = trader_api.list_assets(status='active', asset_class='us_equity')
        assets = [x for x in assets if x.shortable == True and x.exchange == 'NASDAQ']
        stocks = [x.symbol for x in assets][:SCREENER_NASDAQ_COUNT]

        screened = []
        for st in stocks:
            _stock = CheckStock(st)
            if _stock != {}:
                screened.append(_stock)

        screened = [x for x in screened if abs(x['stop_loss'] - x['take_profit']) > min(x['stop_loss'], x['take_profit']) * TAKE_PROFIT_DELTA]
        return screened
    except Exception as e:
        logging.error(f"Error screening stocks: {str(e)}")
        return []

def Trade(api, stock, operation, shares_to_trade, take_profit, stop_loss):
    try:
        api.submit_order(
            symbol=stock,
            qty=shares_to_trade,
            side=operation,
            type='market',
            order_class='bracket',
            time_in_force='day',
            take_profit={'limit_price': take_profit},
            stop_loss={'stop_price': stop_loss}
        )
        message = f'\n\t*{stock}*, qty _{shares_to_trade}_ \n\t\twere {operation}'
        send_message(f'{TRADER_BOT_NAME}: we entered the market with:' + message)
        performance['trades'] += 1
        performance['successful_trades'] += 1
        return True
    except Exception as e:
        logging.error(f"Error executing trade for {stock}: {str(e)}")
        performance['trades'] += 1
        performance['failed_trades'] += 1
        return False

def medium_trader_go(shared_predictions):
    trader_api = api.REST(TRADER_API_KEY, TRADER_API_SECRET, TRADER_API_URL)

    while True:
        try:
            account = trader_api.get_account()
            clock = trader_api.get_clock()
        except Exception as e:
            logging.error(f"Error connecting to Alpaca API: {str(e)}")
            time.sleep(60)
            continue

        if bool(account) == True:
            message = f'''{TRADER_BOT_NAME}: for *{account.account_number}*
            current capital is _{account.portfolio_value}$_
            and non marginable buying power is _{account.non_marginable_buying_power}$_'''
            send_message(message)

        if clock.is_open == True:
            if float(account.non_marginable_buying_power) < CASH_LIMIT:
                message = f"{TRADER_BOT_NAME}: there is no cash on the account or limit reached!"
                send_message(message)
            else:
                screened = ScreenStocks(trader_api)
                if len(screened) > 0:
                    CASH_FOR_TRADE_PER_SHARE = (float(account.non_marginable_buying_power) - CASH_LIMIT) / len(screened)
                    for item in screened:
                        predictions = shared_predictions.get(item['stock'], [])
                        if predictions:
                            STOCK = item['stock']
                            OPERATION = 'buy' if item['direction'] == 'UP' else 'sell'
                            STOP_LOSS = min([item['stop_loss']] + predictions) if item['direction'] == 'UP' else max([item['stop_loss']] + predictions)
                            TAKE_PROFIT = max([item['take_profit']] + predictions) if item['direction'] == 'UP' else min([item['take_profit']] + predictions)
                            SHARE_PRICE = round(min(STOP_LOSS, TAKE_PROFIT), 2)
                            SHARES_TO_TRADE = int(CASH_FOR_TRADE_PER_SHARE / SHARE_PRICE)
                            if abs(STOP_LOSS - TAKE_PROFIT) > SHARE_PRICE * TAKE_PROFIT_DELTA and SHARES_TO_TRADE > 0:
                                Trade(trader_api, STOCK, OPERATION, SHARES_TO_TRADE, TAKE_PROFIT, STOP_LOSS)
                                logging.info(f'{STOCK}: {STOP_LOSS}, {TAKE_PROFIT}, {OPERATION}, {SHARES_TO_TRADE}')

        try:
            portfolio = trader_api.list_positions()
            if bool(portfolio) == True:
                message = f'{TRADER_BOT_NAME}: we have {len(portfolio)} opened positions.'
                total_pl = 0
                for i in portfolio:
                    message += f'\n\t*{i.symbol}*: qty {i.qty} {i.side} for _{i.market_value}$_ \n\t\t\tcurrent price _{i.current_price}$_ \n\t\t\tprofit _{i.unrealized_pl}$_'
                    total_pl += float(i.unrealized_pl)
                performance['total_profit_loss'] = total_pl
                send_message(message)
        except Exception as e:
            logging.error(f"Error fetching portfolio: {str(e)}")

        if clock.is_open == False:
            message = f"{TRADER_BOT_NAME}: the market is *CLOSED*, let's try later on!"
            send_message(message)

        # Log performance metrics
        logging.info(f"Performance metrics: {json.dumps(performance)}")

        time.sleep(300)  # Wait for 5 minutes before next iteration

if __name__ == "__main__":
    prediction_process = Process(target=run_predictions, args=(shared_predictions,))
    trading_process = Process(target=medium_trader_go, args=(shared_predictions,))

    prediction_process.start()
    trading_process.start()

    prediction_process.join()
    trading_process.join()

while True:
    if is_market_open():
        # Perform trading operations
        pass
    else:
        logging.info("Market is currently closed.")

    time.sleep(60)  # Check every minute

Epoch 1/80
94/94 [==============================] - 7s 11ms/step - loss: 0.0154
Epoch 2/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0039
Epoch 3/80
94/94 [==============================] - 1s 11ms/step - loss: 0.0037
Epoch 4/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0031
Epoch 5/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0025
Epoch 6/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0026
Epoch 7/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0026
Epoch 8/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0025
Epoch 9/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0026
Epoch 10/80
94/94 [==============================] - 1s 10ms/step - loss: 0.0020
Epoch 11/80
94/94 [==============================] - 1s 14ms/step - loss: 0.0022
Epoch 12/80
94/94 [==============================] - 2s 16ms/step - loss: 0.0021
Epoch 13/80
94/94 [==================

Process Process-5:
Process Process-6:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-aab0b3ee1254>", line 289, in medium_trader_go
    time.sleep(300)  # Wait for 5 minutes before next iteration
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-aab0b3ee1254>", line 146, in run_predictions
    model = train_model(model, X, Y)
  File "<ipython-input-4-aab0b3ee1254>", line 120, in train_model
    model.fit(X, Y, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/t

KeyboardInterrupt: 